In [5]:
%cd /aiffel/aiffel/aiffelthon/prompt/OpenPrompt/

/aiffel/aiffel/aiffelthon/prompt/OpenPrompt


In [2]:
#!git clone https://github.com/kurbster/Prompt-Summarization.git

Cloning into 'Prompt-Summarization'...
remote: Enumerating objects: 72487, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 72487 (delta 101), reused 189 (delta 100), pack-reused 72297
Receiving objects: 100% (72487/72487), 615.41 MiB | 22.31 MiB/s, done.
Resolving deltas: 100% (18155/18155), done.


In [3]:
#%cd /aiffel/aiffel/aiffelthon/prompt_sum/Prompt-Summarization/

/aiffel/aiffel/aiffelthon/prompt_sum/Prompt-Summarization


In [6]:
!python setup.py install

requirements: ['transformers>=4.10.0', 'sentencepiece==0.1.96', '# scikit-learn>=0.24.2', 'tqdm>=4.62.2', 'tensorboardX', 'nltk', 'yacs', 'dill', 'datasets', 'rouge==1.0.0', 'pyarrow', 'scipy']
running install
/opt/conda/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
running bdist_egg
running egg_info
writing openprompt.egg-info/PKG-INFO
writing dependency_links to openprompt.egg-info/dependency_links.txt
writing requirements to openpr

In [1]:
%cd /aiffel/aiffel/aiffelthon/prompt/OpenPrompt/openprompt

/aiffel/aiffel/aiffelthon/prompt/OpenPrompt/openprompt


In [18]:
!pip install -r requirements.txt

     |████████████████████████████████| 125 kB 6.8 MB/s            
     |████████████████████████████████| 451 kB 64.5 MB/s            
  Attempting uninstall: datasets
    Found existing installation: datasets 1.14.0
    Uninstalling datasets-1.14.0:
      Successfully uninstalled datasets-1.14.0


# 1단계 작업 정의

- 첫 번째 단계는 현재 NLP 작업을 결정하고 데이터가 어떻게 생겼는지, 데이터에서 원하는 것이 무엇인지 생각하는 것입니다! 즉, 이 단계의 본질은 작업의 classses와 를 결정하는 InputExample것입니다. 단순화를 위해 감정 분석을 예로 사용합니다.

In [1]:
!pip install transformers==4.20.1

     |████████████████████████████████| 4.4 MB 6.7 MB/s            
     |████████████████████████████████| 6.6 MB 65.5 MB/s            
     |████████████████████████████████| 182 kB 81.3 MB/s            
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.19
    Uninstalling huggingface-hub-0.0.19:
      Successfully uninstalled huggingface-hub-0.0.19
  Attempting uninstall: transformers
    Found existing installation: transformers 4.11.3
    Uninstalling transformers-4.11.3:
      Successfully uninstalled transformers-4.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 1.14.0 requires huggingface-hub<0.1.0,>=0.0.19, but you have huggingf

In [2]:
import transformers
print(transformers.__version__)

4.20.1


In [3]:
import pandas as pd

In [4]:
# 데이터 불러오기

train_df = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train_total.csv')

In [5]:
# drop columns

train_df.drop(columns = ['Id', 'Category'], axis = 1, inplace = True)

In [6]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9-.,!?]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [7]:
# 전체 Text 데이터에 대한 전처리 (1)
from tqdm import tqdm
import re
clean_text = []

for s in tqdm(train_df['Text']):
    clean_text.append(preprocess_sentence(s))
    
train_df['Text'] = clean_text

# 전체 headlines 데이터에 대한 전처리 
clean_headlines = []

for s in tqdm(train_df['Summary']):
      clean_headlines.append(preprocess_sentence(s))
        
train_df['Summary'] = clean_headlines

100%|██████████| 279992/279992 [00:03<00:00, 91038.91it/s]


In [15]:
from openprompt.data_utils import InputExample

dataset = [ # For simplicity, there's only two examples
    # text_a is the input text of the data, some other datasets may have multiple input sentences in one example.
    InputExample(
        guid = 0,
        text_a = train_df['Text'][0],
        text_b = train_df['Summary'][0]
    ),
    InputExample(
        guid = 1,
        text_a = train_df['Text'][1],
        text_b = train_df['Summary'][1]
    ),
]

# 2단계. PLM 확보

- 작업을 지원하는 PLM을 선택하십시오. 모델마다 속성이 다르므로 OpenPrompt를 사용하여 다양한 PLM의 잠재력을 탐색하는 것이 좋습니다. OpenPrompt는 huggingface의 모델과 호환되며 다음 모델이 테스트되었습니다.

- 마스킹된 언어 모델(MLM): BERT, RoBERTa,ALBERT
- 자기회귀 언어 모델(LM): GPT,GPT2
- 시퀀스 대 시퀀스 모델(Seq2Seq):T5

### 발생한 오류

- ImportError: cannot import name 'OPTConfig' from 'transformers' (/opt/conda/lib/python3.9/site-packages/transformers/__init__.py)
- 해결방법 : transformers version == 4.20.1로 수정
- link : https://github.com/huggingface/transformers/issues/17790

In [22]:
# openprompt에서 사용할 수 있는 Model 및 토크나이저

from transformers import BertConfig, BertTokenizer, BertModel, BertForMaskedLM, \
                         RobertaConfig, RobertaTokenizer, RobertaModel, RobertaForMaskedLM, \
                         AlbertTokenizer, AlbertConfig, AlbertModel, AlbertForMaskedLM, \
                         T5Config, T5Tokenizer, T5ForConditionalGeneration, \
                         OpenAIGPTTokenizer, OpenAIGPTLMHeadModel, OpenAIGPTConfig, \
                         GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, \
                         ElectraConfig, ElectraForMaskedLM, ElectraTokenizer

### 기존에 사용하던 Sk-KoGPT-2의 토크나이저가 적용이 되지 않아 kodialogpt2로 바꿔서 적용해보기로 하였다. 해당 Model은 대화문 data를 학습시킨 Model이다

In [8]:
from openprompt.plms import load_plm
plm, tokenizer, model_config, WrapperClass = load_plm("gpt2", "taeminlee/kodialogpt2-base")


Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 3단계. 템플릿 정의

- A Template는 원본 입력 텍스트의 수정자이며 프롬프트 학습에서 가장 중요한 모듈 중 하나이기도 합니다. 템플릿을 정의하는 고급 자습서는 템플릿 을 작성하는 방법에 있습니다.

- 다음은 the가 in <text_a>으로 대체되고 the 가 레이블 단어를 예측하는 데 사용되는 예입니다.text_aInputExample<mask>

In [31]:
from openprompt.prompts import PtuningTemplate
promptTemplate = PtuningTemplate(
    model = plm,
    tokenizer = tokenizer,
    prompt_encoder_type = 'lstm',
    text = '{"placeholder":"text_a"} summarization {"mask":"text_b"}',
    placeholder_mapping = {'<text_a>': 'text_a', '<text_b>': 'text_b'})

# 4단계. Verbalizer 정의

- A Verbalizer는 프롬프트 학습에서 또 다른 중요하지만(생성에서와 같이 필요하지는 않음) 원래 레이블(우리는 classes레이블 단어 집합으로 정의했습니다. 기억하십니까?)을 투사합니다. Verbalizer 를 정의하는 고급 자습서 는 Verbalizer를 작성하는 방법에 있습니다.

- project the negative class to the word bad

- project the positive class to the words good, wonderful, great.

In [25]:
from openprompt.prompts import GenerationVerbalizer
promptVerbalizer = GenerationVerbalizer(
    tokenizer = tokenizer,
    #classes = classes,
    is_rule = False,)

# 5단계. PromptModel 구성

- 작업이 주어지면 이제 우리는 a PLM, a Template및 a Verbalizer를 가지고 있습니다. 우리는 그것들을 a로 결합합니다 PromptModel.

- 이 예제는 순진하게 세 개의 모듈을 결합하지만 실제로 이들 사이에서 몇 가지 복잡한 상호 작용을 정의할 수 있습니다.

In [26]:
from openprompt import PromptModel
promptModel = PromptModel(
    template = promptTemplate,
    plm = plm,
    #verbalizer = promptVerbalizer,
)

# 6단계. DataLoader 정의

- A 는 기본적으로 a 및 a PromptDataLoader를 포함하는 pytorch Dataloader의 프롬프트 버전입니다 .TokenizerTemplate

In [27]:
from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset = dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
    tokenizer_wrapper_class=WrapperClass,
)

tokenizing: 2it [00:00, 93.69it/s]


# 7단계. 학습 및 추론

- 완료! Pytorch에서 다른 프로세스와 동일하게 학습 및 추론을 수행할 수 있습니다.

In [32]:
import torch

# making zero-shot inference using pretrained MLM with prompt
promptModel.eval()
with torch.no_grad():
    for batch in data_loader:
        logits = promptModel(batch)
        preds = torch.argmax(logits, dim = -1)
        print(tokenizer.decode(batch['input_ids'][0], skip_special_tokens=True))
# predictions would be 1, 0 for classes 'positive', 'negative'

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not CausalLMOutputWithCrossAttentions

In [29]:
import torch

# making zero-shot inference using pretrained MLM with prompt
promptModel.eval()
with torch.no_grad():
    for batch in data_loader:
        logits = promptModel(batch)
        preds = torch.argmax(logits, dim = -1)
        print(classes[preds])

TypeError: argmax(): argument 'input' (position 1) must be Tensor, not CausalLMOutputWithCrossAttentions